In [ ]:
pip install matplotlib

In [ ]:
# 📈 Analyse Crypto en Temps Réel — CoinGecko

import requests
import csv
import time
from datetime import datetime
import matplotlib.pyplot as plt

# ================================
# 1️⃣ CONFIGURATION
# ================================

CRYPTOS    = ["bitcoin", "ethereum", "solana"]
INTERVALLE = 30       # secondes entre chaque collecte
FICHIER    = "crypto_data.csv"

# Historique des prix pour le graphique
historique_temps   = []
historique_bitcoin  = []
historique_ethereum = []
historique_solana   = []

# Créer l'en-tête du CSV
with open(FICHIER, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["heure", "bitcoin", "ethereum", "solana"])

print("Démarrage — Ctrl+C pour arrêter\n")

# ================================
# 2️⃣ BOUCLE PRINCIPALE
# ================================

while True:

    # -- Appel API --
    url      = "https://api.coingecko.com/api/v3/simple/price?ids=bitcoin,ethereum,solana&vs_currencies=usd&include_24hr_change=true"
    reponse  = requests.get(url, timeout=10)
    donnees  = reponse.json()

    heure    = datetime.now().strftime("%H:%M:%S")
    btc      = donnees["bitcoin"]["usd"]
    eth      = donnees["ethereum"]["usd"]
    sol      = donnees["solana"]["usd"]
    btc_var  = donnees["bitcoin"]["usd_24h_change"]
    eth_var  = donnees["ethereum"]["usd_24h_change"]
    sol_var  = donnees["solana"]["usd_24h_change"]

    # -- Affichage console --
    print(f"{'=' * 45}")
    print(f"  {heure}")
    print(f"  BTC  ${btc:>10,.2f}   ({btc_var:+.2f}%)")
    print(f"  ETH  ${eth:>10,.2f}   ({eth_var:+.2f}%)")
    print(f"  SOL  ${sol:>10,.2f}   ({sol_var:+.2f}%)")

    # -- Sauvegarde CSV --
    with open(FICHIER, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([heure, btc, eth, sol])
    print(f"   Sauvegardé")

    # -- Mise à jour historique graphique --
    historique_temps.append(heure)
    historique_bitcoin.append(btc)
    historique_ethereum.append(eth)
    historique_solana.append(sol)

    # Garder seulement les 20 derniers points
    if len(historique_temps) > 20:
        historique_temps.pop(0)
        historique_bitcoin.pop(0)
        historique_ethereum.pop(0)
        historique_solana.pop(0)

    # -- Graphique --
    plt.clf()                              # efface le graphique précédent
    plt.plot(historique_temps, historique_bitcoin,  label="Bitcoin",  color="#F7931A", marker='o')
    plt.plot(historique_temps, historique_ethereum, label="Ethereum", color="#627EEA", marker='o')
    plt.plot(historique_temps, historique_solana,   label="Solana",   color="#9945FF", marker='o')

    plt.title("Prix Crypto en Temps Réel (USD)")
    plt.xlabel("Heure")
    plt.ylabel("Prix (USD)")
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.pause(0.1)                         # affiche le graphique

    # -- Attente --
    print(f"  ⏱️  Prochaine collecte dans {INTERVALLE}s...")
    time.sleep(INTERVALLE)